In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/14 15:02:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/14 15:02:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
time_zone = "Europe/Rome" # America/New_York
time_zone_utc = "UTC"

datetime_format = "yyyy-MM-dd HH:mm"
windows_size = "1 hour"

spark.conf.set('spark.sql.session.timeZone', time_zone)

In [4]:
data_df = spark.createDataFrame([
    (1, "2023-03-25 23:00"),
    (1, "2023-03-26 00:00"),
    (1, "2023-03-26 01:00"),
    (1, "2023-03-26 01:59"),
    # (1, "2023-03-26 02:00"),
    (1, "2023-03-26 03:00"),
    (1, "2023-03-26 04:00"),
    (1, "2023-03-26 10:00"),
], schema=["id", "datetime"])

#data_df = data_df.withColumn("datetime", F.to_timestamp(F.col("datetime"), "yyyy-MM-dd HH:mm"))
data_df.show(truncate=False)

+---+----------------+
|id |datetime        |
+---+----------------+
|1  |2023-03-25 23:00|
|1  |2023-03-26 00:00|
|1  |2023-03-26 01:00|
|1  |2023-03-26 01:59|
|1  |2023-03-26 03:00|
|1  |2023-03-26 04:00|
|1  |2023-03-26 10:00|
+---+----------------+



In [5]:
data1_df = data_df.withColumn("unix_time", F.unix_timestamp("datetime", datetime_format)).drop("datetime")
data1_df.show()

+---+----------+
| id| unix_time|
+---+----------+
|  1|1679781600|
|  1|1679785200|
|  1|1679788800|
|  1|1679792340|
|  1|1679792400|
|  1|1679796000|
|  1|1679817600|
+---+----------+



In [6]:
spark.conf.set("spark.sql.session.timeZone", time_zone_utc)

data1_df = data1_df.withColumn("datetime_utc", F.from_unixtime("unix_time", datetime_format))
data1_df.show(truncate=False)

+---+----------+----------------+
|id |unix_time |datetime_utc    |
+---+----------+----------------+
|1  |1679781600|2023-03-25 22:00|
|1  |1679785200|2023-03-25 23:00|
|1  |1679788800|2023-03-26 00:00|
|1  |1679792340|2023-03-26 00:59|
|1  |1679792400|2023-03-26 01:00|
|1  |1679796000|2023-03-26 02:00|
|1  |1679817600|2023-03-26 08:00|
+---+----------+----------------+



In [7]:
data1_df = data1_df.withColumn("window_utc", F.window(timeColumn=F.col("datetime_utc"), windowDuration=windows_size))
data1_df.show(truncate=False)

+---+----------+----------------+------------------------------------------+
|id |unix_time |datetime_utc    |window_utc                                |
+---+----------+----------------+------------------------------------------+
|1  |1679781600|2023-03-25 22:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|
|1  |1679785200|2023-03-25 23:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|
|1  |1679788800|2023-03-26 00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|
|1  |1679792340|2023-03-26 00:59|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|
|1  |1679792400|2023-03-26 01:00|{2023-03-26 01:00:00, 2023-03-26 02:00:00}|
|1  |1679796000|2023-03-26 02:00|{2023-03-26 02:00:00, 2023-03-26 03:00:00}|
|1  |1679817600|2023-03-26 08:00|{2023-03-26 08:00:00, 2023-03-26 09:00:00}|
+---+----------+----------------+------------------------------------------+



23/07/14 15:02:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
data1_df = data1_df.withColumn("window_start_utc", F.col("window_utc").start)
data1_df = data1_df.withColumn("window_end_utc", F.col("window_utc").end)
data1_df.show(truncate=False)

+---+----------+----------------+------------------------------------------+-------------------+-------------------+
|id |unix_time |datetime_utc    |window_utc                                |window_start_utc   |window_end_utc     |
+---+----------+----------------+------------------------------------------+-------------------+-------------------+
|1  |1679781600|2023-03-25 22:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|2023-03-25 22:00:00|2023-03-25 23:00:00|
|1  |1679785200|2023-03-25 23:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|2023-03-25 23:00:00|2023-03-26 00:00:00|
|1  |1679788800|2023-03-26 00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|2023-03-26 00:00:00|2023-03-26 01:00:00|
|1  |1679792340|2023-03-26 00:59|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|2023-03-26 00:00:00|2023-03-26 01:00:00|
|1  |1679792400|2023-03-26 01:00|{2023-03-26 01:00:00, 2023-03-26 02:00:00}|2023-03-26 01:00:00|2023-03-26 02:00:00|
|1  |1679796000|2023-03-26 02:00|{2023-03-26 02:00:00, 2023-03-2

In [12]:
data1_df = data1_df.withColumn("window_start", F.from_utc_timestamp(F.col("window_start_utc"), time_zone))
data1_df = data1_df.withColumn("window_end", F.from_utc_timestamp(F.col("window_end_utc"), time_zone))
data1_df.show(truncate=False)

+---+----------+----------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|id |unix_time |datetime_utc    |window_utc                                |window_start_utc   |window_end_utc     |window_start       |window_end         |
+---+----------+----------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|1  |1679781600|2023-03-25 22:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|2023-03-25 22:00:00|2023-03-25 23:00:00|2023-03-25 23:00:00|2023-03-26 00:00:00|
|1  |1679785200|2023-03-25 23:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|2023-03-25 23:00:00|2023-03-26 00:00:00|2023-03-26 00:00:00|2023-03-26 01:00:00|
|1  |1679788800|2023-03-26 00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|2023-03-26 00:00:00|2023-03-26 01:00:00|2023-03-26 01:00:00|2023-03-26 03:00:00|
|1  |1679792340|2023-03-26 00:59|{2023-03-26 00:00:00, 202

23/07/14 20:16:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 961534 ms exceeds timeout 120000 ms
23/07/14 20:16:00 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/14 20:31:18 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

## Appunti

- Ho capito che bisogna fare il bucketing impostando la time zone a UTC altrimenti i bucket vengono fuori non uniformi
- PySpark non ha DateTimeType che includano le TimeZone
- PySpark ha una TimeZone impostata di default e per ogni DateTimeType che ha utilizza implicitamente quella